# Instruction of programming assignments for AMS 380: Data Mining



We will use the Python programming language for all assignments in this course. Specifically, we will use a few popular libraries (numpy, matplotlib, math) for scientific computing.

Python & Numpy Tutorial
-----------------------
- Official Python tutorial: https://docs.python.org/3/tutorial/
- Official Numpy tutorial: https://numpy.org/doc/
- Good tutorial sources: https://cs231n.github.io/python-numpy-tutorial/


Dataset Descriptions
--------------------
We will use USPS dataset for this assignments. The USPS dataset is in the “data” folder: USPS.mat. The whole data has already been loaded into the matrix A. The matrix A contains all the images of size 16 × 16. Each of the 3000 rows in A corresponds to the image of one handwritten digit (between 0 and 9).


Assignment Descriptions
-----------------------
There are total three blocks including 'Main', 'Solution' and 'Helper'. In this assignment, you only need to add your solution under 'Solution' file following the given instruction. However, you might need to read all the files to fully understand the requirement.

The 'Helper' block includes all the helper functions for the assignments, like load data, show images, etc. The 'Main' block is used to test your solution.

Notes: Do not change anything in 'Main' and 'Helper' blocks (except directory path: data_loc). Only try to add your code to 'Solution' block and keep function names and parameters unchanged.


Helper functions
----------------
In this assignment, you may want to use following helper functions:
- np.linalg.svd(): compute the singular value decomposition on a given matrix.
- np.dot(): matrices multiplication operation.
- np.mean(): compute the mean value on a given matrix.
- np.ones(): generate a all '1' matrix with a certain shape.
- np.transpose(): matrix transpose operation.
- np.linalg.norm(): compute the norm value of a matrix. You may use it for the reconstruct_error function.

# Helper

In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.io as scio

In [7]:
def load_data(dataloc):
  data = scio.loadmat(dataloc)
  return data['A']


def show_images(data, p, i):
    fig = plt.figure()
    data = data
    plt.title('Reconstructed image%d with %d component' % (i, p))
    plt.imshow(data[i,:].reshape((16,16)))
    fig.savefig('Reconstructed image%d with %d component'%(i,p))

# Solution

In [8]:
class PCA():
    def __init__(self, X, n_components):
        '''
        Args:
            X: The data matrix of shape [n_samples, n_features].
            n_components: The number of principal components. A scaler number.
        '''

        self.n_components = n_components
        self.X = X
        self.Up, self.Xp = self._do_pca()


    def _do_pca(self):
        '''
        To do PCA decomposition.
        Returns:
            Up: Principal components (transform matrix) of shape [n_features, n_components].
            Xp: The reduced data matrix after PCA of shape [n_samples, n_components].
        '''
        ### YOUR CODE HERE

        n_samples, n_features = self.X.shape

        # Get X_centered = X[i] - X_bar
        X_bar = np.mean(self.X, axis=0)

        X_centered = np.empty((self.X.shape))
        for i in range(n_samples):
          X_centered[i] = self.X[i] - X_bar

        X_centered = X_centered.T  # ---> Do transpose because shape should be (p, n) and do SDV. With (n, p), U in SDV is different

        # Get matrix U in SDV of X_centered, and take k columns to be Up(transform matrix), shape = [p, k]
        U, D, V = np.linalg.svd(X_centered)
        Up = U[:,:self.n_components] # [n_features, n_components]

        # Xp = Up.T * X_centered
        Xp = np.dot(Up.T, X_centered).T # [n_components, n_samples] ---> [n_samples, n_components]

        return Up, Xp

        ### END YOUR CODE

    def get_reduced(self, X=None):
        '''
        To return the reduced data matrix.
        Args:
            X: The data matrix with shape [n_any, n_features] or None.
               If None, return reduced training X.
        Returns:
            Xp: The reduced data matrix of shape [n_any, n_components].
        '''
        if X is None:
            return self.Xp
        else:
            return X@self.Up

    def reconstruction(self, Xp):
        '''
        To reconstruct reduced data given principal components Up.

        Args:
        Xp: The reduced data matrix after PCA of shape [n_samples, n_components].

        Return:
        X_re: The reconstructed matrix of shape [n_samples, n_features].
        '''
        ### YOUR CODE HERE

        n_samples, n_features = self.X.shape
        X_bar = np.zeros((n_features,))
        for i in range(n_samples):
          X_bar += self.X[i] / n_samples


        re_X_centered = np.dot(self.Up, Xp.T)


        X_re = np.empty((re_X_centered.shape))

        row, col = X_re.shape
        for i in range(col):
          X_re[:,i] = re_X_centered[:,i] + X_bar

        X_re = X_re.T # [n_features, n_samples] --->[n_samples, n_features]

        return X_re

        ### END YOUR CODE


def reconstruct_error(A, B):
    '''
    To compute the reconstruction error.

    Args:
    A & B: Two matrices needed to be compared with. Should be of same shape.

    Return:
    error: the Frobenius norm's square of the matrix A-B. A scaler number.
    '''
    ### YOUR CODE HERE

    return "Error:", np.linalg.norm(A - B, 'fro')

    ### END YOUR CODE


# Main

In [9]:
def test_pca():
    dataloc = "../data/USPS.mat"
    A = load_data(dataloc)
    ks = [10, 50, 100, 200]
    for k in ks:
      pca = PCA(A, k)
      Ak = pca.get_reduced()
      A_re = pca.reconstruction(Ak)
      error = reconstruct_error(A, A_re)
      print(error)
      show_images(A_re, k, 0)
      show_images(A_re, k, 1)

if __name__ == '__main__':
	test_pca()

FileNotFoundError: [Errno 2] No such file or directory: '../data/USPS.mat'